# Load Data

In [1]:
import pandas as pd
df=pd.read_json('dblp-ref-3.json',lines=True)
df


MemoryError: 

# Pull links 

In [4]:
import re
import numpy as np

year=1990
linkslist=[]

while(year<2018):
    t=df[df['year']<=year]
    print('Length of',year,'data is',len(t))

    ftdic2={}
    #re.findall(r'\'([0-9a-z-]+)\'',t1['References'][0])
    for index,row in t.iterrows():
    
        if isinstance(row['references'],list):
            refs=row['references']
        else:
            refs=[]
            
        for i in refs:
            if ftdic2.get(i,0) ==0:
                ftdic2[i]=[]
                ftdic2[i].append(row['id'])
            else:
                ftdic2[i].append(row['id'])

    fromid2=[]
    toid2=[]
    for (i,j) in ftdic2.items():
        for ids in j:
            toid2.append(i)
            fromid2.append(ids)

    linksdf1=pd.DataFrame()
    linksdf1['From']=fromid2
    linksdf1['To']=toid2
    incid=[]
    t=linksdf1.copy()
    t.columns=['To','From']
    mdf=pd.merge(linksdf1,t,how='inner',left_on='From',right_on='From')

    incid=list(mdf['From'].unique())


    links=linksdf1[ (linksdf1['To'].isin(incid) | linksdf1['From'].isin(incid))]
    links=links.reset_index().drop(['index'],axis=1)
    print('Links length',len(links))
    linkslist.append(links)
    year+=1
    

Length of 1990 data is 627
Links length 3
Length of 1991 data is 785
Links length 25
Length of 1992 data is 876
Links length 38
Length of 1993 data is 992
Links length 38
Length of 1994 data is 1172
Links length 55
Length of 1995 data is 1318
Links length 76
Length of 1996 data is 1512
Links length 77
Length of 1997 data is 1758
Links length 88
Length of 1998 data is 1967
Links length 91
Length of 1999 data is 2226
Links length 113
Length of 2000 data is 2581
Links length 130
Length of 2001 data is 2966
Links length 130
Length of 2002 data is 3794
Links length 134
Length of 2003 data is 4537
Links length 191
Length of 2004 data is 5575
Links length 257
Length of 2005 data is 6473
Links length 335
Length of 2006 data is 8382
Links length 446
Length of 2007 data is 8881
Links length 482
Length of 2008 data is 9345
Links length 584
Length of 2009 data is 9864
Links length 681
Length of 2010 data is 10446
Links length 745
Length of 2011 data is 11021
Links length 805
Length of 2012 data is

In [ ]:
Page rank

In [5]:
pagerank_list=[]

for link_i in range(len(linkslist)) :
    unique_ids=list( set( list(linkslist[link_i]['From']) + list(linkslist[link_i]['To']) ) )
    page_rank=pd.DataFrame()
    page_rank['Paper Id']=pd.Series(unique_ids)
    page_rank['Old rank']=np.zeros(len(unique_ids))
    page_rank['New rank']=np.ones(len(unique_ids))


# Get latest page ranks for strongly connected component
    prev_ranks = dict()
    prev_ranks = dict(zip(page_rank['Paper Id'],page_rank['New rank']))

    many = 1

# Sanity check
    if len(prev_ranks) < 1 : 
        print("Nothing to page rank.  Check data.")

# Lets do Page Rank in memory so it is really fast
    myt=dict()
    for i in range(many):
        # print prev_ranks.items()[:5]
        next_ranks = dict();
        myt=prev_ranks
        total = 0.0
        total=sum(prev_ranks.values())
        next_ranks = dict( zip(page_rank['Paper Id'],np.zeros( len(page_rank['Paper Id']) ) ) )
        # print total
    
    
        # Find the number of outbound links and sent the page rank down each
        for (node, old_rank) in list(prev_ranks.items()):
            # print node, old_rank
            give_ids = list()
            temp_give=list(linkslist[link_i][linkslist[link_i]['From']==node]['To'].values)
            give_ids=temp_give
            #for ttid in temp_give:
                #if ttid in next_ranks.keys():
                    #give_ids.append(ttid)
        
            if ( len(give_ids) < 1 ) : continue
        
            amount = old_rank / len(give_ids)
        # print node, old_rank,amount, give_ids
    
            for id in give_ids:
                next_ranks[id] = next_ranks[id] + amount
    
        newtot = 0
        newtot=sum(next_ranks.values())
        evap = (total - newtot) / len(next_ranks)
    
    # print newtot, evap
        for node in next_ranks:
            next_ranks[node] = next_ranks[node] + evap
    
        newtot = 0
        newtot=sum(next_ranks.values())
        # Compute the per-page average change from old rank to new rank
        # As indication of convergence of the algorithm
        totdiff = 0
        for (node, old_rank) in list(prev_ranks.items()):
            new_rank = next_ranks[node]
            diff = abs(old_rank-new_rank)
            totdiff = totdiff + diff
    
        avediff = totdiff / len(prev_ranks)
        print(i+1, avediff)
        # rotate
        prev_ranks = next_ranks
    pagerank=pd.DataFrame()
    pagerank['Paper Id']=list(myt.keys())
    pagerank['Old rank']=list(myt.values())
    pagerank['New rank']=list(next_ranks.values())
    pagerank_list.append(pagerank)
    # Put the final ranks back into the database
    print(list(next_ranks.items())[:5])

    

1 0.25
[('514567f3-be9c-42a7-adee-65adb6b1ecf5', 0.5), ('96975e30-cb5c-499b-acad-4139df969cef', 1.0), ('2bd1873a-184f-49d6-a2e3-6456e178b442', 1.5), ('23e87727-a573-4cbc-ad44-179a13fcf1b1', 1.0)]
1 0.288888888889
[('f0a5e86a-36b0-4c47-b085-0d35f15e9e8e', 1.1666666666666665), ('376bc7b9-d3ef-4fac-b81f-47fe6b930c88', 0.75757575757575746), ('5627c121-516b-429b-ab9a-856372724938', 1.1666666666666665), ('8ee2a9c1-902b-4b9c-b42d-8e6ffd004057', 0.66666666666666652), ('3801766c-d5fd-4c4c-8349-e511a20eb8c4', 0.75757575757575746)]
1 0.268595041322
[('f0a5e86a-36b0-4c47-b085-0d35f15e9e8e', 1.2272727272727271), ('376bc7b9-d3ef-4fac-b81f-47fe6b930c88', 0.81818181818181801), ('5627c121-516b-429b-ab9a-856372724938', 1.2272727272727271), ('8ee2a9c1-902b-4b9c-b42d-8e6ffd004057', 0.72727272727272707), ('3801766c-d5fd-4c4c-8349-e511a20eb8c4', 0.81818181818181801)]
1 0.268595041322
[('f0a5e86a-36b0-4c47-b085-0d35f15e9e8e', 1.2272727272727271), ('376bc7b9-d3ef-4fac-b81f-47fe6b930c88', 0.81818181818181801),

1 0.149830541554
[('b45038a2-f1e4-4bab-9070-b373af10b1d4', 0.91163363896283278), ('cd45ddd3-35a8-4057-83de-c901f472b960', 0.93300116033035407), ('f6a95614-14e2-4b52-98ec-a60f16765e27', 0.89056084202796681), ('1f73723c-b904-4d93-8045-d8de3772fb27', 0.91490161281904192), ('af269f01-707e-480f-9021-45e0a3f75c08', 0.92274475007394385)]
1 0.129707224889
[('a54c87ea-3122-4fea-adca-c2292506e61f', 0.9539101153187719), ('b45038a2-f1e4-4bab-9070-b373af10b1d4', 0.93254259395125061), ('092e668a-c74f-462c-81c2-e5ba462fbe3e', 0.97698703839569501), ('cd45ddd3-35a8-4057-83de-c901f472b960', 0.9539101153187719), ('cbb20d95-ee90-4132-bf2d-a608f0eb25e6', 1.876987038395695)]


In [6]:
pagerank_list[0].head()

,Paper Id,Old rank,New rank
0,514567f3-be9c-42a7-adee-65adb6b1ecf5,1.0,0.5
1,96975e30-cb5c-499b-acad-4139df969cef,1.0,1.0
2,2bd1873a-184f-49d6-a2e3-6456e178b442,1.0,1.5
3,23e87727-a573-4cbc-ad44-179a13fcf1b1,1.0,1.0


# Visualization of network

In [ ]:


print("Creating JSON output on spider.js...")
dft=df.copy()

for link_i in range(20,len(linkslist)-2) :
    notfound=[]
    pids=list(pagerank_list[link_i]['Paper Id'])
    fname='spider'+str(link_i)+'.js'
    fhand = open(fname,'w')
    nodes = list()
    maxrank = None
    minrank = None
    fcnt=0
    for row in pids :
        c=len(linkslist[link_i][linkslist[link_i]['To']==node]['From'])
        olr=pagerank_list[link_i][pagerank_list[link_i]['Paper Id']==row]['Old rank'].values[0]
        ner=pagerank_list[link_i][pagerank_list[link_i]['Paper Id']==row]['New rank'].values[0]
        url=row
        try:
            turl=dft[dft['id']==row]['venue'].values[0]
            if len(turl)>1:
                url=turl
                fcnt+=1
                #print('Found')
        except:
            #print('Venue not found')
            notfound.append(row)

        id=row
        li=(c,olr,ner,id,url)
    
        nodes.append(li)
        rank = ner
        if maxrank is None or maxrank < rank: maxrank = rank
        if minrank is None or minrank > rank : minrank = rank
        if len(nodes) > len(pagerank_list[link_i]) : break
    if maxrank == minrank or maxrank is None or minrank is None:
        print("Error - please run sprank.py to compute page rank")
 

    fhand.write('spiderJson = {"nodes":[\n')
    count = 0
    map = dict()
    ranks = dict()
    for row in nodes :
        if count > 0 : fhand.write(',\n')
        # print row
        rank = row[2]
        rank = 19 * ( (rank - minrank) / (maxrank - minrank) ) 
        fhand.write('{'+'"weight":'+str(row[0])+',"rank":'+str(rank)+',')
        fhand.write(' "id":"'+str(row[3])+'" , "url":"'+row[4]+'"}')
        map[row[3]] = count
        ranks[row[3]] = rank
        count = count + 1
    fhand.write('],\n')


    fhand.write('"links":[\n')

    count = 0
    fincnt=0
    for index,row in linksdf1.iterrows() :
        # print row
        if row['From'] not in map or row['To'] not in map : continue
        if count > 0 : fhand.write(',\n')
        rank = ranks[row['From']]
        srank = 19 * ( (rank - minrank) / (maxrank - minrank) ) 
        fhand.write('{"source":'+str(map[row['From']])+',"target":'+str(map[row['To']])+',"value":3}')
        count = count + 1
        fincnt+=1
        #print('in')
    fhand.write(']};')

    print('fincnt :',fincnt)
    print('Found count :',fcnt)

# Eliminating unknown links, droping unwanted cols, and author count calculation

In [7]:
pro_data=[]
for link_i in range(len(pagerank_list)):
    temp=pd.merge(df.set_index('id'),pagerank_list[link_i].set_index('Paper Id'),how='inner',left_index=True,right_index=True)
    temp.drop(['abstract','references','title'],axis=1,inplace=True)
    temp=temp[temp['venue'].str.len()>1]
    temp['author_count']=temp.apply(lambda x: len(x['authors']),axis=1)
    #temp['authors']=temp['authors'].astype(str)
    temp.reset_index(inplace=True)
    temp=temp.rename(index=str, columns={'index':'id'})
    pro_data.append(temp)
#temp[temp['New rank']==max(temp['New rank'])]


# author cols creation 

In [8]:
def cutauth(x,l):
    if len(x)>l : 
        return x[l]
    else : 
        return np.nan

for j in range(len(pro_data)):
    for l in range(3):
        st='author '+str(l)
        pro_data[j][st]=pro_data[j]['authors'].apply(cutauth,args=(l,))
pro_data[4].head()

,id,authors,venue,year,Old rank,New rank,author_count,author 0,author 1,author 2
0,514567f3-be9c-42a7-adee-65adb6b1ecf5,[Gary E. Kopec],"international conference on acoustics, speech,...",1984,1.0,0.737705,1,Gary E. Kopec,NaN,NaN
1,2bd1873a-184f-49d6-a2e3-6456e178b442,[Gary E. Kopec],"international conference on acoustics, speech,...",1983,1.0,1.737705,1,Gary E. Kopec,NaN,NaN
2,fef89403-e7f7-45fa-8da9-6375fc06dc5c,"[Mari Ostendorf, Salim Roukos]","IEEE Transactions on Acoustics, Speech, and Si...",1989,1.0,1.737705,2,Mari Ostendorf,Salim Roukos,NaN
3,27318145-8f25-4c7d-8dcb-c45cb7699bda,"[Joseph Polifroni, Stephanie Seneff, Victor W....",north american chapter of the association for ...,1991,1.0,0.737705,3,Joseph Polifroni,Stephanie Seneff,Victor W. Zue
4,8ee2a9c1-902b-4b9c-b42d-8e6ffd004057,"[Colin W. Wightman, N. M. Veilleuz, Mari Osten...",north american chapter of the association for ...,1991,1.0,0.737705,3,Colin W. Wightman,N. M. Veilleuz,Mari Ostendorf


# age  calculation

In [9]:
#pro_data[1]['authors']=pro_data[1]['authors'].astype(str)
#pro_data[1][pro_data[1]['authors']==pro_data[1]['authors'][49]]
yc=1991

pro_data[0]['age']=pro_data[0].apply(lambda x: abs(x['year']-1990),axis=1)
pro_data[0]['pub_year']=pro_data[0].apply(lambda x: x['year'],axis=1)
pro_data[0]['year_data']=1990
final_data=pro_data[0]

for lnk_i in range(1,len(pro_data)):
    pro_data[lnk_i]['age']=pro_data[lnk_i].apply(lambda x: abs(x['year']-yc),axis=1)
    pro_data[lnk_i]['pub_year']=pro_data[lnk_i].apply(lambda x: x['year'],axis=1)
    pro_data[lnk_i]['year_data']=yc
    yc+=1
    final_data=final_data.append(pro_data[lnk_i], ignore_index=True)
len(final_data)
final_data.head()

,id,authors,venue,year,Old rank,New rank,author_count,author 0,author 1,author 2,age,pub_year,year_data
0,514567f3-be9c-42a7-adee-65adb6b1ecf5,[Gary E. Kopec],"international conference on acoustics, speech,...",1984,1.0,0.500000,1,Gary E. Kopec,NaN,NaN,6,1984,1990
1,2bd1873a-184f-49d6-a2e3-6456e178b442,[Gary E. Kopec],"international conference on acoustics, speech,...",1983,1.0,1.500000,1,Gary E. Kopec,NaN,NaN,7,1983,1990
2,514567f3-be9c-42a7-adee-65adb6b1ecf5,[Gary E. Kopec],"international conference on acoustics, speech,...",1984,1.0,0.666667,1,Gary E. Kopec,NaN,NaN,7,1984,1991
3,2bd1873a-184f-49d6-a2e3-6456e178b442,[Gary E. Kopec],"international conference on acoustics, speech,...",1983,1.0,1.666667,1,Gary E. Kopec,NaN,NaN,8,1983,1991
4,fef89403-e7f7-45fa-8da9-6375fc06dc5c,"[Mari Ostendorf, Salim Roukos]","IEEE Transactions on Acoustics, Speech, and Si...",1989,1.0,1.666667,2,Mari Ostendorf,Salim Roukos,NaN,2,1989,1991


# Assiging integer values to predict using random forest

In [10]:
final_data['id'] = final_data['id'].astype('category')
final_data['authors']=final_data['authors'].astype(str)
final_data['venue'] = final_data['venue'].astype('category')
final_data['authors'] = final_data['authors'].astype('category')
final_data['author 0'] = final_data['author 0'].astype('category')
final_data['author 1'] = final_data['author 1'].astype('category')
final_data['author 2'] = final_data['author 2'].astype('category')
remid=dict( enumerate(final_data['id'].cat.categories) )
cat_columns =final_data.select_dtypes(['category']).columns
final_data[cat_columns] = final_data[cat_columns].apply(lambda x: x.cat.codes)
#final_data.drop('authors',inplace=True,axis=1)
final_data.head()

,id,authors,venue,year,Old rank,New rank,author_count,author 0,author 1,author 2,age,pub_year,year_data
0,1111,932,553,1984,1.0,0.500000,1,838,-1,-1,6,1984,1990
1,571,932,553,1983,1.0,1.500000,1,838,-1,-1,7,1983,1990
2,1111,932,553,1984,1.0,0.666667,1,838,-1,-1,7,1984,1991
3,571,932,553,1983,1.0,1.666667,1,838,-1,-1,8,1983,1991
4,3550,1749,170,1989,1.0,1.666667,2,1575,2205,-1,2,1989,1991


In [11]:
features=final_data.copy()
#features.drop('Old rank',axis=1,inplace=True)
features.drop('pub_year',axis=1,inplace=True)
#features.drop('authors',axis=1,inplace=True)
features.drop('year',axis=1,inplace=True)
#features.drop('author 0',axis=1,inplace=True)
#features.drop('author 1',axis=1,inplace=True)
#features.drop('author 2',axis=1,inplace=True)
#features.drop('author_count',axis=1,inplace=True)
#features.drop('venue',axis=1,inplace=True)
#features.drop('age',axis=1,inplace=True)
features.head()

,id,authors,venue,Old rank,New rank,author_count,author 0,author 1,author 2,age,year_data
0,1111,932,553,1.0,0.500000,1,838,-1,-1,6,1990
1,571,932,553,1.0,1.500000,1,838,-1,-1,7,1990
2,1111,932,553,1.0,0.666667,1,838,-1,-1,7,1991
3,571,932,553,1.0,1.666667,1,838,-1,-1,8,1991
4,3550,1749,170,1.0,1.666667,2,1575,2205,-1,2,1991


# Test & train data seperation

In [12]:
import numpy as np
# Labels are the values we want to predict
features.drop(['Old rank'],axis=1,inplace=True)
labels = np.array(features['New rank'])
#test_labels = np.array( features['New rank'][ (features['age']>11) & (features['year_data']==2017) ] )
# Remove the labels from the features
# axis 1 refers to the columns
#test_features= features[ (features['age']>11) & (features['year_data']==2017) ].drop('New rank', axis = 1)
#testdf=test_features
#traindf=features

#train_labels = np.array(features['New rank'][ features['year_data']<2017 ] )
#train_features= features[ features['year_data']<2017 ].drop('New rank', axis = 1)
features= features.drop('New rank',axis=1)

# Saving feature names for later use
feature_list = list(features.columns)
# Convert to numpy array

#features = np.array(features)
#test_features=np.array(test_features)
#len(features)
features.head()

,id,authors,venue,author_count,author 0,author 1,author 2,age,year_data
0,1111,932,553,1,838,-1,-1,6,1990
1,571,932,553,1,838,-1,-1,7,1990
2,1111,932,553,1,838,-1,-1,7,1991
3,571,932,553,1,838,-1,-1,8,1991
4,3550,1749,170,2,1575,2205,-1,2,1991


In [13]:
from sklearn.model_selection import train_test_split
# Split the data into training and testing sets
train_features, test_features, train_labels, test_labels = train_test_split(features, labels, test_size = 0, random_state = 42)

print('Training Features Shape:', train_features.shape)
print('Training Labels Shape:', train_labels.shape)
print('Testing Features Shape:', test_features.shape)
print('Testing Labels Shape:', test_labels.shape)

Training Features Shape: (5353, 9)
Training Labels Shape: (5353,)
Testing Features Shape: (0, 9)
Testing Labels Shape: (0,)


# Random forest algorithm

In [14]:
# Import the model we are using
from sklearn.ensemble import RandomForestRegressor
# Instantiate model with 1000 decision trees
rf = RandomForestRegressor(n_estimators =4000, random_state = 42)
# Train the model on training data
rf.fit(train_features,train_labels);

In [ ]:
# Use the forest's predict method on the test data
predictions = rf.predict(test_features)
print(predictions)
# Calculate the absolute errors
errors = abs(predictions - test_labels)
# Print out the mean absolute error (mae)
#print('Mean Absolute Error:', round(np.mean(errors), 2), 'degrees.')

In [ ]:
# Calculate mean absolute percentage error (MAPE)
mape = 100 * (errors / test_labels)
# Calculate and display accuracy
accuracy = 100 - np.mean(mape)
print('Accuracy:', round(accuracy, 2), '%.')

#  Gradient boosting

In [ ]:
from sklearn.metrics import mean_squared_error,r2_score
from sklearn.ensemble import GradientBoostingRegressor

params = {'n_estimators': 3500, 'max_depth': 6,'learning_rate': 0.06, 'loss': 'huber','alpha':0.95}

clf = GradientBoostingRegressor(**params).fit(train_features,train_labels)

#pre=clf.predict(test_features)
#print(pre)
#errors = abs( clf.predict(test) - test)
#mape = 100 * (errors / test)
# Calculate and display accuracy
#accuracy = 100 - np.mean(mape)
#print('Accuracy:', round(accuracy, 2), '%.')

In [ ]:
#final_data['id']=final_data['id'].astype('category')
#final_data['id']=final_data['id'].cat.codes
#print(list(zip(final_data['id'].cat.codes,final_data['id'].cat.categories)))
errors = abs( pre - test_labels)
mape = 100 * (errors / test_labels)
# Calculate and display accuracy
accuracy = 100 - np.mean(mape)
print('Accuracy:', round(accuracy, 2), '%.')

# Multiple regression 

In [ ]:
from scipy import stats, integrate
import statsmodels.api as sm
#lab=features['New rank']
#features.drop(['New rank','Old rank'],axis=1,inplace=True)
features.head()

In [ ]:
#X = features[['id','venue','age','year_data','author 0','author 1','author 2']] #chose three variables (one of which is a dummy variable)
#y = lab
model = sm.OLS(train_labels,train_features).fit() 
predictions = model.predict(test_features)

In [ ]:
# Calculate the absolute errors
errors = abs(predictions - test_labels)
# Print out the mean absolute error (mae)
print('Mean Absolute Error:', round(np.mean(errors), 2), 'degrees.')
# Calculate mean absolute percentage error (MAPE)
mape = 100 * (errors / test_labels)
# Calculate and display accuracy
accuracy = 100 - np.mean(mape)
print('Accuracy:', round(accuracy, 2), '%.')

#  Plot 

In [ ]:
import matplotlib.pyplot as plt

pre=clf.predict(test_features)
#pre=rf.predict(test_features)
#pre=model.predict(test_features)

pr=list(pre)
ac=list(test_labels)
yr=list(test_features['year_data'])
pr.sort()
ac.sort()
yr.sort()

plt.plot(ac, yr, label='actual')
plt.plot(predictions, yr, label='predicted')

plt.xlabel('x label')
plt.ylabel('y label')

plt.title("Simple Plot")

plt.legend()
plt.show()

In [ ]:
#1536	726	413	3	654	1994	170	7	2016
#id	authors	venue	author_count	author 0	author 1	author 2	age	year_data
te=pd.DataFrame()
te['id']=[1536]
te['authors']=[726]
te['venue']=[413]
te['author_count']=[3]
te['author 0']=[654]
te['author 1']=[1994]
te['author 2']=[170]
te['age']=[12]
te['year_data']=[2021]
print(rf.predict(te))

In [15]:
import pickle
#pickle.dump(rf, open('rf_predict', 'wb'))
#pickle.dump(clf, open('gba_predict', 'wb'))
#clf.predict(test)
train_features.head()

,id,authors,venue,author_count,author 0,author 1,author 2,age,year_data
5010,1200,2511,411,2,2259,2529,-1,0,2017
1650,2185,2197,560,5,1979,2705,1932,16,2016
401,1986,549,631,3,497,1799,1284,15,2006
4222,1240,1090,593,2,983,641,-1,0,2017
1684,1536,726,413,3,654,1994,170,7,2016


In [20]:
xy=2025
#future_test=train_features.copy()
#uq=list(future_test['id'].unique())
#future_test=future_test[ (future_test['id'].isin(uq)) & (future_test['year_data']==2017) ]
#future_test.to_pickle('test_feature')
#df.to_pickle('all_data')

future_test = pd.read_pickle('test_feature')
df = pd.read_pickle('all_data')

future_test['year_data']=xy
future_test['age']+=(xy-2017)

rf=pickle.load(open('rf_predict', 'rb'))
#print(len(future_test))
future_test.head()
#future_test.count()

,id,authors,venue,author_count,author 0,author 1,author 2,age,year_data
5010,1200,2511,411,2,2259,2529,-1,8,2025
4222,1240,1090,593,2,983,641,-1,8,2025
3265,1526,523,98,1,471,-1,-1,8,2025
5325,1631,363,289,3,325,2750,1401,8,2025
3235,3077,3000,371,2,2700,1370,-1,8,2025


In [21]:
#pd.Series(remid)
#remid[1200]
from heapq import nlargest
fin=future_test.copy()
fin['Rank']=rf.predict(future_test)
t=[]
for i in nlargest(20,list(fin['Rank'])):
    t.append(fin['id'][fin['Rank']==i].values[0])
print(t)


[2316, 2319, 2898, 2961, 2727, 3430, 2801, 3516, 2314, 2669, 2317, 2323, 3180, 2318, 2322, 3464, 3331, 2315, 2321, 2320]


In [22]:
#pd.Series(remid).iloc[1200]
idli=[]
for i in t:
    idli.append(remid[i])
df['venue'][df['id'].isin(idli)].values

array(['global communications conference', 'web search and data mining',
       'global communications conference', 'IEEE Access',
       'Digital Signal Processing',
       'Genomics, Proteomics & Bioinformatics',
       'human factors in computing systems',
       'arXiv: Computer Vision and Pattern Recognition',
       'arXiv: Computer Vision and Pattern Recognition',
       'vehicular technology conference', 'Computers in Human Behavior',
       'vehicular technology conference',
       'vehicular technology conference',
       'Wireless Personal Communications',
       'Foundations of Computational Mathematics',
       'International Journal of Computer Vision',
       'Computers & Industrial Engineering',
       'neural information processing systems',
       'international conference on data engineering',
       'australasian conference on information security and privacy'], dtype=object)